# "THE PRICE IS RIGHT"
build a model that predicts how much something costs from a description, based on a scrape of Amazon data


A model that can estimate how much something costs, from its description.

## STEP 4: Neural Networks and LLMs

In [1]:
# imports

import os
from dotenv import load_dotenv
from huggingface_hub import login
from pricer.evaluator import evaluate
from litellm import completion
from pricer.items import Item
import numpy as np
from tqdm.notebook import tqdm
import csv
from sklearn.feature_extraction.text import HashingVectorizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import CosineAnnealingLR


In [2]:
LITE_MODE = True

load_dotenv(override=True)
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
username = "<username>"
dataset = f"{username}/items_lite" if LITE_MODE else f"{username}/items_full"

train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Loaded 20,000 training items, 1,000 validation items, 1,000 test items


Saving as Dataset. This could have been done in previous notebook winstead of uploading to hub.

In [8]:
from datasets import Dataset, DatasetDict

ds_local = DatasetDict({
    "train": Dataset.from_list([item.model_dump() for item in train]),
    "validation": Dataset.from_list([item.model_dump() for item in val]),
    "test": Dataset.from_list([item.model_dump() for item in test]),
})
ds_local.save_to_disk("data_local") 

Saving the dataset (0/1 shards):   0%|          | 0/20000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving as CSV Files

In [10]:
import pandas as pd
os.makedirs('data_local_csv', exist_ok=True)  
for name, items in [("train", train), ("val", val), ("test", test)]:
    df = pd.DataFrame([item.model_dump() for item in items])
    df.to_csv(f"data_local_csv/{name}.csv", index=False)

To Load Data

In [ ]:
from datasets import Dataset, DatasetDict

ds_local = DatasetDict({
    "train": Dataset.from_list([item.model_dump() for item in train]),
    "validation": Dataset.from_list([item.model_dump() for item in val]),
    "test": Dataset.from_list([item.model_dump() for item in test]),
})
ds_local.save_to_disk("data_local")  # Creates folder with arrow files

# a vanilla Neural Network

In [11]:
# Prepare our documents and prices

y = np.array([float(item.price) for item in train])
documents = [item.summary for item in train]

# Creating a Bag of Words or Vocabulary Vector for all the words
- The Vectorizer will fit the documents to vocabulary and then that will be the input size to the neural network

In [12]:
# Use the HashingVectorizer for a Bag of Words model
# Using binary=True with the CountVectorizer makes "one-hot vectors"

np.random.seed(42)
vectorizer = HashingVectorizer(n_features=5000, stop_words='english', binary=True)
X = vectorizer.fit_transform(documents)

In [37]:
# Define the neural network - 8 layer neural network

class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 64)
        self.layer4 = nn.Linear(64, 64)
        self.layer5 = nn.Linear(64, 64)
        self.layer6 = nn.Linear(64, 64)
        self.layer7 = nn.Linear(64, 64)
        self.layer8 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)  # or 0.3 — typical range 0.1–0.5

    def forward(self, x):
        output1 = self.dropout(self.relu(self.layer1(x)))
        output2 = self.dropout(self.relu(self.layer2(output1)))
        output3 = self.dropout(self.relu(self.layer3(output2)))
        output4 = self.dropout(self.relu(self.layer4(output3)))
        output5 = self.dropout(self.relu(self.layer5(output4)))
        output6 = self.dropout(self.relu(self.layer6(output5)))
        output7 = self.dropout(self.relu(self.layer7(output6)))
        output8 = self.layer8(output7)   # No dropout before output
        return output8

Dropout randomly turns off some neurons during training (sets their outputs to 0). It’s used to reduce overfitting.
How it works
During training:
Each neuron has probability p (e.g. 0.2) of being dropped.
Dropped neurons output 0 and don’t contribute to the next layer.
The remaining outputs are scaled (e.g. by 1/(1-p)) so the overall activation level stays similar.
Which neurons are dropped changes each mini-batch.
During evaluation/inference: Dropout is turned off. All neurons are used so predictions are stable.

In [ ]:
# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X.toarray())
y_train_tensor = torch.FloatTensor(y).unsqueeze(1)
 ## .unsqueeze(1) Helps to rotate the array and make it 2d but with one column and N rows, instead of N Columns 1 Row
 ## as we map the features in X rows with Y in index. Pytorch expects this way

In [ ]:
X_train_tensor

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [26]:
y_train_tensor

tensor([[ 64.3000],
        [ 79.0000],
        [240.0000],
        ...,
        [ 50.5600],
        [ 35.0000],
        [189.9900]])

In [27]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_tensor, y_train_tensor, test_size=0.01, random_state=42)

# Create the loader, Create Pytorch Dataset and then use DataLoader that is a utility by pytorch to load data efficiently
# #  using batches while training
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [28]:
train_dataset

In [ ]:
X_train_tensor.shape ## 20,000 Samples or rows and we selected 5000 Parameters or features in HashVectorizing

torch.Size([20000, 5000])

In [38]:
# Initialize the model
input_size = X_train_tensor.shape[1] ## 1 index is Input/features size that is a Bag of words here
model = NeuralNetwork(input_size)

In [31]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Number of trainable parameters: {trainable_params:,}")

Number of trainable parameters: 669,249


### Training

In [ ]:
# Define loss function and optimizer

loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# We will do 2 complete runs through the data

EPOCHS = 2

for epoch in range(EPOCHS):
    model.train()
    for batch_X, batch_y in tqdm(train_loader):
        optimizer.zero_grad()

        # The next 4 lines are the 4 stages of training: forward pass, loss calculation, backward pass, optimize
        
        outputs = model(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = loss_function(val_outputs, y_val)

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {loss.item():.3f}, Val Loss: {val_loss.item():.3f}')

  0%|          | 0/310 [00:00<?, ?it/s]

Epoch [1/5], Train Loss: 4615.657, Val Loss: 18490.104


  0%|          | 0/310 [00:00<?, ?it/s]

Epoch [2/5], Train Loss: 10089.146, Val Loss: 18530.018


  0%|          | 0/310 [00:00<?, ?it/s]

Epoch [3/5], Train Loss: 12340.702, Val Loss: 18411.512


  0%|          | 0/310 [00:00<?, ?it/s]

Epoch [4/5], Train Loss: 3192.434, Val Loss: 19065.061


  0%|          | 0/310 [00:00<?, ?it/s]

Epoch [5/5], Train Loss: 993.934, Val Loss: 17122.588


In [35]:
def neural_network(item):
    model.eval()
    with torch.no_grad():
        vector = vectorizer.transform([item.summary])
        vector = torch.FloatTensor(vector.toarray())
        result = model(vector)[0].item()
    return max(0, result)

In [36]:
evaluate(neural_network, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$128 $148 $14 $3 $36 $123 $9 $105 $48 $219 $468 $293 $50 $551 $15 $3 $18 $2 $132 $138 $37 $36 $92 $286 $100 $294 $342 $58 $97 $33 $39 $122 $78 $21 $90 $98 $32 $20 $111 $134 $152 $32 $39 $10 $90 $62 $36 $71 $4 $45 $2 $40 $13 $46 $128 $78 $31 $121 $46 $32 $119 $38 $37 $12 $491 $126 $17 $26 $71 $262 $10 $30 $165 $79 $13 $40 $214 $48 $30 $42 $74 $103 $34 $35 $11 $117 $89 $138 $25 $142 $28 $43 $5 $8 $88 $101 $12 $110 $117 $101 $44 $43 $24 $29 $5 $18 $98 $268 $17 $152 $38 $28 $56 $22 $114 $333 $261 $161 $70 $4 $21 $431 $128 $13 $46 $82 $20 $180 $168 $84 $8 $88 $153 $101 $72 $26 $93 $35 $48 $137 $68 $165 $71 $256 $183 $83 $82 $400 $98 $6 $10 $6 $11 $18 $17 $28 $111 $5 $33 $2 $81 $5 $15 $31 $362 $9 $91 $29 $22 $30 $7 $28 $95 $23 $7 $27 $7 $14 $24 $16 $279 $7 $244 $61 $4 $27 $26 $6 $39 $10 $92 $5 $29 $63 $20 $35 $186 $87 $10 $19 

#### This is so far the best since we have 5000 features to deciside and with 8 layers we have 600k + Parameters that we can change and Grad Decent, RElu etc etc helps to optimise this

## LLM Methods

In [42]:
def messages_for(item):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}"
    return [{"role": "user", "content": message}]

In [39]:
print(test[0].summary)

Title: Excess V2 Distortion/Modulation Pedal  
Category: Music Pedals  
Brand: Old Blood Noise  
Description: A versatile pedal offering distortion and three modulation modes—delay, chorus, and harmonized fifths—with full control over signal routing and expression.  
Details: Features include separate gain, tone, and volume controls; time, depth, and volume per modulation; order switching, soft‑touch bypass, and expression jack for dynamic control.


In [40]:
# The function for gpt-4.1-nano

def gpt_4__1_nano(item):
    response = completion(model="openai/gpt-4.1-nano", messages=messages_for(item))
    return response.choices[0].message.content

In [43]:
gpt_4__1_nano(test[0])

'$250'

In [44]:
test[0].price

219.0

#### As Noted Above, the prediction is off by 31$ the loss is definitely better than our own model

In [47]:
len(test)

1000

In [53]:
evaluate(gpt_4__1_nano, test, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

$31 $34 $25 $20 $20 $80 $6 $95 $11 $870 

### Trying the latest big model

In [54]:
# The function for gpt-5.1

def gpt_5__2(item):
    response = completion(model="gpt-5.2", messages=messages_for(item), reasoning_effort='high', seed=42)
    return response.choices[0].message.content

In [56]:
evaluate(gpt_5__2, test, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

$30 $34 $25 $10 $20 $100 $54 $35 $6 $1 